In [ ]:
from os.path import join
from numpy import array,load,save,reshape,linspace,meshgrid,pi,stack,cos,sin
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.pyplot import contourf,figure,scatter,gca,show
from pandas import read_csv
from matplotlib.collections import LineCollection

Dat = "path/to/data"

Grid = array(read_csv(join(Dat,"an-grid.csv")))
VorZ = load(join(Dat,"an-vorticity.npy"))
print(VorZ.max(),VorZ.min())

print(Grid.shape,VorZ.shape)

Fig = figure(figsize=(5,5))

x = Grid[:,0]
y = Grid[:,1]

eps = 1e-3

rad = linspace(0.25,0.5,64)
tet = linspace(0,2.0*pi,128)

rmat,tmat = meshgrid(rad,tet)

xi = 0.5 + rmat * cos(tmat)
yi = 0.5 + rmat * sin(tmat)

scatter(xi,yi)
segs1 = stack((xi,yi), axis = 2)
segs2 = segs1.transpose(1, 0, 2)
gca().add_collection(LineCollection(segs1))
gca().add_collection(LineCollection(segs2))
show()

print(xi,yi)


In [ ]:
zi = []
for i in range(140):
  zz = griddata((x,y),VorZ[:,i],(xi,yi),method='nearest')#,fill_value=0)
  print(zz.min(),zz.max())
  zi.append(zz)
  print("Sample %d"%i)

Case = array(zi)
print(Case.shape)
print( "Min Value = ",Case.min())
print( "Max Value = ",Case.max())
save(join(Dat,"an-VorZ-interp2d.npy"),Case)

In [ ]:
plt.style.use("dark_background")
Fig = figure(figsize=(5,5))

#ccf = contourf(xi,yi,zi[9],cmap="jet",levels=linspace(0,1,10),vmin=0,vmax=1)
ccf = contourf(xi,yi,zi[118],cmap="jet",levels=linspace(-1,1,10),vmin=-1,vmax=1)
Fig.show()

In [ ]:
from os.path import join
from numpy import reshape,newaxis,where,nan,load
Dat = "path/to/data"

Case = load(join(Dat,"an-VorZ-interp2d.npy"))[:,:,:,newaxis]
print(Case.shape)
print(Case.max())

In [263]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
import tensorflow as tf
from keras.optimizers import adam_v2
from keras.layers import LeakyReLU

#config = tf.compat.v1.ConfigProto(device_count={"CPU": 8})
#tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

Padmet = "same"

Acmet1 = "relu"
Acmet2 = "linear"
Acmet3 = LeakyReLU(alpha=0.1)

# Input Layer
input = Input(shape=(128, 64, 1))  

cs1 = 7
cs2 = 3
ms  = 2

# Encoder Part
x = Conv2D(64, (cs1, cs1), padding=Padmet, activation=Acmet3)(input)
x = MaxPooling2D((ms, ms), padding=Padmet)(x)
x = Conv2D(32, (cs1, cs1), padding=Padmet, activation=Acmet3)(x)
x = MaxPooling2D((ms, ms), padding=Padmet)(x)
x = Conv2D(16, (cs2, cs2), padding=Padmet, activation=Acmet3)(x)
x = MaxPooling2D((ms, ms), padding=Padmet)(x)
x = Conv2D(16, (cs2, cs2), padding=Padmet, activation=Acmet3)(x)
enc = MaxPooling2D((ms, ms), padding=Padmet)(x)

# Decoder Part
x = Conv2D(16, (cs2, cs2), padding=Padmet, activation=Acmet3)(enc)
x = UpSampling2D((ms, ms))(x)
x = Conv2D(16, (cs2, cs2), padding=Padmet, activation=Acmet3)(x)
x = UpSampling2D((ms, ms))(x)
x = Conv2D(32, (cs1, cs1), padding=Padmet, activation=Acmet3)(x)
x = UpSampling2D((ms, ms))(x)
x = Conv2D(64, (cs1, cs1), padding=Padmet, activation=Acmet3)(x)
x = UpSampling2D((ms, ms))(x)
output = Conv2D(1, (cs1, cs1), padding=Padmet, activation=Acmet3)(x)

autoencoder = Model(input, output)
#autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

#opt = tf.optimizers.Adam(learning_rate = 1e-3)
#pt = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.99)
#opt = adam_v2.Adam(learning_rate=1e-3,decay=1e-4/1000)

opt = tf.optimizers.Adam(learning_rate = 1e-3)


autoencoder.compile(optimizer=opt, loss='mse')

autoencoder.summary()

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_31 (InputLayer)       [(None, 128, 64, 1)]      0         
                                                                 
 conv2d_270 (Conv2D)         (None, 128, 64, 64)       3200      
                                                                 
 max_pooling2d_120 (MaxPooli  (None, 64, 32, 64)       0         
 ng2D)                                                           
                                                                 
 conv2d_271 (Conv2D)         (None, 64, 32, 32)        100384    
                                                                 
 max_pooling2d_121 (MaxPooli  (None, 32, 16, 32)       0         
 ng2D)                                                           
                                                                 
 conv2d_272 (Conv2D)         (None, 32, 16, 16)        462

In [ ]:
from keras.callbacks import LearningRateScheduler
from math import pow,floor

# learning rate schedule
def step_decay(epoch):
	initial_lrate = 0.0001
	#drop = 0.5
	#epochs_drop = 2500 #250
	#lrate = initial_lrate * pow(drop,floor((1+epoch)/epochs_drop))
	lrate = initial_lrate 
	return lrate

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [ ]:
from keras.models import load_model,Model
autoencoder = load_model("VORZ-AE-7-3-ADAM-1")
'''
hist = autoencoder.fit(Case,
                       Case,
                       epochs=2000,
                       callbacks=callbacks_list,
                       validation_data=(Case,Case))
                       #batch_size=int(Case.shape[0]/8))

from matplotlib.pyplot import plot,xscale,yscale

xscale("log")
yscale("log")
plot(hist.history["loss"])
show()
'''

In [ ]:
from numpy import savetxt
savetxt("Loss-AE-7-3-2.txt",array(hist.history["loss"]))
savetxt("Val-Loss-AE-7-3-2.txt",array(hist.history["val_loss"]))

In [ ]:
from sklearn.metrics import mean_squared_error
pred = autoencoder.predict(Case)

avg = 0
for i in range(140):
    mse = mean_squared_error(Case[i,:,:,0],pred[i,:,:,0])
    avg += mse
avg /=140
print("AVERAGED MSE Error = %.4e" % avg)

In [ ]:
yc = Case[51][:,:,0]
yp = pred[51][:,:,0]
print("Precidtion max = ",yp.max(),", min = ",yp.min())
print(yc.shape,yp.shape)

In [ ]:
from matplotlib.pyplot import figure,show,contourf
Fig = figure(figsize=(5,5))
ccf = contourf(xi,yi,yc,cmap="jet")#,levels=linspace(0,1,10),vmin=0,vmax=1)
Fig.show()

Fig = figure(figsize=(5,5))
ccf = contourf(xi,yi,yp,cmap="jet")#,levels=linspace(0,1,10),vmin=0,vmax=1)
Fig.show()

Fig = figure(figsize=(5,5))
ccf = contourf(xi,yi,yp-yc,cmap="jet")#,levels=linspace(0,1,10),vmin=0,vmax=1)
plt.colorbar(cmap="jet")
Fig.show()

In [ ]:
#from keras.models import load_model,Model
#autoencoder.save("VORZ-AE-7-3-ADAM-3")
#a2 = load_model("VORZ-AE-7-3-ADAM-3")
#print(a2)